# HW3: Part 3
## Information Retrival
---




### Read Data

In [1]:
import csv

In [10]:
def read_data_from_csv(file_path):
    with open(file_path, 'r') as file:
        content = list(csv.DictReader(file))
        return content

In [11]:
dataset = read_data_from_csv('./dataset.csv')

### Show First 10

In [12]:
def show_first_ten_titles(doc_ids):
    print('Top 10 results:')
    for d in doc_ids[:10]:
        print(dataset[d]['title'])

### Preprocess

In [13]:
from hazm import *

In [14]:
def remove_punctuations(text):
    return text.replace(".", '').replace("،", "").replace(":", "").replace("؛", "").replace("!", "").replace("؟", "")

In [15]:
def preprocess_dataset(dataset):
    normalizer = Normalizer()
    new_dataset = dataset.copy()
    stop_words = stopwords_list()
    stemmer = Stemmer()
    for data in new_dataset:
        text = data['title'] + " " + data['body']
        normalized_text = normalizer.normalize(text)
        only_text = remove_punctuations(normalized_text)
        preprocessed_tokens = word_tokenize(only_text)
        postprocessed_tokens = []
        for token in preprocessed_tokens:
            if token not in stop_words:
                postprocessed_tokens.append(stemmer.stem(token))
        data['tokens'] = postprocessed_tokens
    return new_dataset

In [16]:
def get_all_unique_tokens_in_dataset(dataset):
    all_tokens = []
    for data in dataset:
        all_tokens += data['tokens']
    return list(set(all_tokens))

In [17]:
processed_dataset = preprocess_dataset(dataset)

In [18]:
tokens = get_all_unique_tokens_in_dataset(processed_dataset)

### Boolean Retrival

#### Create Doc-Term Matrix

In [34]:
def get_doc_term_matrix(tokens, dataset):
    tokens_num = len(tokens)
    dataset_size = len(dataset)
    tokens_hash = {tokens[i]: i for i in range(tokens_num)}
    df_dict = {f'Doc{i}': [0] * tokens_num for i in range(dataset_size)}

    for i in range(dataset_size):
        for token in dataset[i]['tokens']:
            current_hash = tokens_hash[token]
            df_dict[f'Doc{i}'][current_hash] = 1
    return df_dict, tokens_hash

In [35]:
term_doc_matrix, tokens_hash = get_doc_term_matrix(tokens, processed_dataset)

In [36]:
import pandas as pd

In [37]:
data_frame = pd.DataFrame(term_doc_matrix)
data_frame.head()

,Doc0,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8,Doc9,...,Doc5333,Doc5334,Doc5335,Doc5336,Doc5337,Doc5338,Doc5339,Doc5340,Doc5341,Doc5342
0,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Boolean Model

In [38]:
class BooleanModel:
    def __init__(self, doc_term_matrix, words_hash):
        self.doc_term_matrix = doc_term_matrix
        self.words_hash = words_hash
        self.normalizer = Normalizer().normalize
        self.stemmer = Stemmer().stem
        self.dataset_size = len(doc_term_matrix.keys())

    def parse_query(self, query):
        words = query.split()
        words[0] = self.normalizer(words[0])
        words[2] = self.normalizer(words[2])
        
        words[0] = self.stemmer(words[0])
        words[2] = self.stemmer(words[2])
        return words
        
    
    def get_word_index(self, word):
        return self.words_hash[word]
    def do_operation(self, operator, operand1, operand2):
        op1_index = self.get_word_index(operand1)
        op2_index = self.get_word_index(operand2)
        results = set()
        if operator == 'and':
            for i in range(self.dataset_size):
                if self.doc_term_matrix[f"Doc{i}"][op1_index] == 1 and self.doc_term_matrix[f"Doc{i}"][op2_index] == 1:
                    results.add(i)
        elif operator == 'or':
            for i in range(self.dataset_size):
                if not (self.doc_term_matrix[f"Doc{i}"][op1_index] == 0 and self.doc_term_matrix[f"Doc{i}"][op2_index] == 0):
                    results.add(i)
        else:
            raise Exception('Unsupported operation')
        return results
    def printSimilars(self, query):
        parsed_query = self.parse_query(query)
        lhs, rhs, operator = None, None, None
        results = set()
        for word in parsed_query:
            if lhs is None:
                lhs = word
            elif operator is None:
                operator = word
            elif rhs is None:
                rhs = word
            else:
                res = self.do_operation(operator, lhs, rhs)
                results = results.union(res)
                lhs = word
                operator = None
                rhs = None

        res = self.do_operation(operator, lhs, rhs)
        results = results.union(res)
        return list(results)

In [39]:
booleanModel = BooleanModel(data_frame, tokens_hash)

##### Example 1

In [40]:
word1 = 'آمریکا'
word2 = 'دولت'
operation = 'and'
query = f"{word1} {operation} {word2}"
result = booleanModel.printSimilars(query)
print(f"{len(result)} docs found: \n{result}")
show_first_ten_titles(result)


180 docs found: 
[512, 3, 1028, 4617, 12, 527, 1040, 17, 2577, 1556, 1046, 1531, 1048, 1053, 1060, 37, 1062, 1063, 4646, 43, 1069, 5165, 51, 1076, 1078, 571, 1084, 1596, 63, 575, 1088, 70, 1094, 72, 73, 1095, 1096, 1101, 1103, 3666, 1107, 86, 1626, 1115, 94, 606, 1120, 1121, 99, 100, 4711, 1128, 1130, 110, 112, 627, 628, 122, 635, 636, 1146, 1150, 3709, 3206, 648, 651, 142, 1690, 166, 168, 683, 174, 686, 177, 689, 690, 691, 700, 704, 4291, 202, 203, 714, 725, 733, 224, 4320, 1762, 742, 1770, 747, 751, 1776, 4339, 768, 770, 4355, 775, 264, 265, 778, 779, 270, 787, 4374, 281, 283, 795, 296, 297, 809, 301, 304, 307, 819, 309, 2869, 311, 4917, 2363, 317, 320, 4424, 845, 335, 1362, 853, 4951, 856, 2904, 346, 858, 348, 860, 862, 2908, 354, 868, 872, 2416, 369, 4979, 4981, 381, 385, 5001, 5003, 912, 401, 2960, 404, 2457, 414, 415, 416, 417, 936, 427, 946, 435, 436, 442, 2496, 449, 963, 453, 455, 969, 460, 1487, 464, 5073, 981, 982, 490, 1517, 501, 1018, 507, 1532]
Top 10 results:
بعضی‌ها نامه

##### Eample 2

In [41]:
word1 = 'حمله'
word2 = 'اسرائیل'
operation = 'and'
query = f"{word1} {operation} {word2}"
result = booleanModel.printSimilars(query)
print(f"{len(result)} docs found: \n{result}")
show_first_ten_titles(result)

21 docs found: 
[512, 2624, 1094, 658, 789, 853, 281, 2908, 669, 415, 735, 425, 809, 812, 559, 1008, 4273, 1136, 501, 1142, 636]
Top 10 results:
بعضی‌ها نامه نوشتند و از رهبری خواستند تا با آمریکا کنار بیاییم
جزئیات جدید از تیراندازی در سفارت آذربایجان | اعترافات متهم ؛ روایت یک شاهد
۲۰۲۲؛ سال تغییر ائتلاف‌ها در خاورمیانه | از بازگشت آمریکا به منطقه تا روی کارآمدن تندروها در تل آویو و شکل‌گیری نسل جدید مبارزان فلسطینی
روایت رئیس موساد از یک اقدام مخفیانه ایران | وحشت بی‌سابقه اسرائیل از معامله تسلیحات پیشرفته میان ایران و روسیه
هشدار مقام ایرانی: اگر برای آغاز یک جنگ منطقه‌ای تصمیم بگیرند ... | اسرائیل می‌داند که ایران مثل گذشته به حمله اصفهان پاسخ می‌دهد 
برای پیگیری دیپلماسی عمومی ۱۰ نفر را به امریکا بفرستیم | جمهوری اسلامی موافقان را به جان نثار تبدیل کند | اغتشاشات باز می گردد اگر ... 
جزئیات جدید ترور شهید سردار قاسم سلیمانی | کلمه‌ای که ترامپ را متقاعد کرد!
نظام سلطه امروز ضعیف‌تر از هر زمان دیگری است | افق‌های پیش روی ما روشن است
رایزنی ضد ایرانی مشاوران امنیتی کاخ سفید و تل‌آوی

##### Example 3

In [42]:
word1 = 'برتر'
word2 = 'فوتبالیست'
operation = 'or'
query = f"{word1} {operation} {word2}"
result = booleanModel.printSimilars(query)
print(f"{len(result)} docs found: \n{result}")
show_first_ten_titles(result)

424 docs found: 
[4096, 4109, 4114, 4119, 4120, 2081, 5159, 5171, 5179, 4156, 4159, 2118, 2121, 5193, 5196, 4176, 4188, 4189, 4192, 3180, 5229, 3191, 5245, 5249, 5254, 3209, 4236, 5264, 4245, 150, 5271, 5273, 4250, 3230, 4257, 4273, 5301, 4280, 5304, 2236, 5309, 3268, 1222, 5318, 4296, 3272, 5322, 3278, 4304, 3280, 2258, 3284, 3285, 4311, 3287, 3288, 3289, 3290, 5338, 4317, 3292, 5340, 3295, 4321, 3298, 3299, 3300, 4329, 3307, 3312, 4337, 3313, 3315, 3317, 3321, 3323, 3324, 3325, 3326, 3328, 4354, 3331, 3332, 3335, 3336, 3340, 2317, 3348, 3349, 3350, 3351, 3355, 3356, 3359, 3362, 3365, 296, 3374, 4402, 3378, 3380, 3383, 3385, 1339, 3388, 3393, 3395, 3397, 3398, 4426, 4429, 3406, 3411, 1365, 3413, 4441, 3417, 3418, 3422, 2404, 3429, 3435, 3436, 3437, 3438, 3441, 3443, 3444, 2422, 4471, 4472, 3449, 378, 3451, 3452, 3457, 3460, 4486, 4489, 3467, 3470, 399, 3471, 3475, 3476, 3477, 3478, 3481, 1434, 3482, 3483, 3491, 3492, 3497, 2485, 4537, 3515, 1468, 2496, 3520, 3521, 454, 3528, 3529, 353

### TF-IDF

In [19]:
from collections import defaultdict
from math import log10

In [20]:
normalizer = Normalizer()
stop_words = stopwords_list()
stemmer = Stemmer()

In [21]:
def preprocess_query(query):
    normalized_query = normalizer.normalize(query)
    only_text = remove_punctuations(normalized_query)
    preprocessed_tokens = word_tokenize(only_text)
    postprocessed_tokens = []
    for token in preprocessed_tokens:
        if token not in stop_words:
            postprocessed_tokens.append(stemmer.stem(token))
    return postprocessed_tokens

In [46]:
class TF_IDF:
    def __init__(self, corpus, threshhold=3):
        self.corpus = corpus
        self.tf, self.df = self.calculate_tf_and_df()
        self.idf = self.calculate_idf()
        self.threshhold = threshhold
        
    def calculate_tf_and_df(self):
        corpus = self.corpus
        df = defaultdict(int)
        tf = []
        for doc in corpus:
            temp_tf = defaultdict(int)
            for term in doc['tokens']:
                temp_tf[term] += 1
                if temp_tf[term] == 1:
                    df[term] += 1
            tf.append(temp_tf)
        return tf, df
    
    def calculate_idf(self):
        corpus, df = self.corpus, self.df
        idf = {}
        corpus_size = len(corpus)
        for term, freq in df.items():
            idf[term] = log10(corpus_size / freq)
        return idf

    def get_score(self, query, index):
        score = 0
        query_tokens = preprocess_query(query)
        for word in query_tokens:
            current_tf = self.tf[index][word]
            score += self.idf[word] * current_tf
        return score
    
    def find_relavent(self, query):
        corpus_size = len(self.corpus)
        score_doc_list = []
        for i in range(corpus_size):
            score = self.get_score(query, i)
            score_doc_list.append((i, score))
        result = []
        for doc_id, score in score_doc_list:
            if score >= self.threshhold:
                result.append(doc_id)
        
        return result

In [47]:
tf_idf = TF_IDF(processed_dataset)

#### Example 1

In [48]:
query = 'تروریسم در ایران'
result = tf_idf.find_relavent(query)
print(f"{len(result)} docs found: \n{result}")
show_first_ten_titles(result)

401 docs found: 
[0, 1, 2, 3, 5, 6, 12, 21, 22, 23, 30, 33, 34, 36, 37, 40, 42, 43, 44, 45, 47, 51, 52, 55, 63, 70, 72, 77, 89, 94, 99, 110, 112, 120, 126, 132, 139, 142, 144, 145, 146, 164, 165, 168, 171, 174, 176, 177, 180, 192, 196, 198, 202, 203, 208, 210, 212, 224, 229, 235, 241, 243, 257, 265, 267, 268, 270, 271, 280, 281, 283, 285, 286, 290, 292, 296, 297, 300, 301, 304, 307, 308, 313, 320, 335, 341, 345, 346, 349, 366, 367, 368, 381, 385, 386, 404, 414, 415, 416, 417, 422, 432, 435, 436, 446, 453, 455, 456, 457, 464, 466, 482, 490, 491, 500, 501, 505, 507, 509, 511, 517, 519, 521, 527, 529, 530, 533, 537, 545, 547, 557, 559, 562, 570, 571, 573, 575, 578, 589, 597, 606, 620, 623, 627, 628, 636, 647, 648, 649, 651, 654, 658, 660, 665, 668, 669, 671, 677, 679, 683, 687, 689, 690, 691, 694, 700, 703, 704, 705, 706, 709, 713, 714, 716, 718, 724, 725, 733, 735, 742, 746, 747, 750, 751, 754, 755, 757, 759, 762, 770, 773, 775, 778, 779, 780, 785, 786, 787, 788, 798, 804, 809, 810, 813,

#### Example 2

In [49]:
query = 'خادم صادق ملت'
result = tf_idf.find_relavent(query)
print(f"{len(result)} docs found: \n{result}")
show_first_ten_titles(result)

214 docs found: 
[7, 14, 17, 51, 52, 89, 112, 115, 122, 132, 168, 265, 318, 349, 367, 382, 415, 416, 421, 435, 456, 468, 491, 505, 507, 510, 518, 519, 529, 547, 557, 570, 606, 628, 672, 694, 701, 718, 724, 754, 816, 823, 872, 945, 946, 969, 982, 986, 989, 1003, 1006, 1021, 1026, 1036, 1039, 1142, 1186, 1216, 1248, 1277, 1303, 1309, 1311, 1323, 1325, 1523, 1542, 1545, 1607, 1690, 1723, 1770, 1805, 1810, 1821, 1875, 1885, 1930, 1938, 1965, 2069, 2114, 2228, 2239, 2329, 2330, 2360, 2366, 2424, 2435, 2437, 2455, 2465, 2547, 2577, 2620, 2730, 2739, 2808, 2814, 3065, 3213, 3244, 3265, 3273, 3274, 3277, 3280, 3305, 3339, 3357, 3367, 3403, 3424, 3433, 3434, 3435, 3451, 3455, 3461, 3489, 3492, 3494, 3510, 3516, 3527, 3536, 3544, 3548, 3555, 3556, 3566, 3568, 3572, 3588, 3624, 3628, 3645, 3650, 3666, 3673, 3689, 3690, 3696, 3709, 3710, 3727, 3734, 3736, 3746, 3784, 3795, 3809, 3817, 3832, 3843, 3860, 3872, 3881, 3902, 3905, 3907, 3908, 3925, 3940, 3955, 3965, 3967, 3971, 3983, 4003, 4067, 4096, 

#### Example 3

In [50]:
query = 'بازیکن فوتبال برتر'
result = tf_idf.find_relavent(query)
print(f"{len(result)} docs found: \n{result}")
show_first_ten_titles(result)

486 docs found: 
[18, 431, 434, 475, 569, 620, 754, 952, 1760, 1790, 1989, 2121, 2360, 2518, 2570, 2909, 3070, 3180, 3264, 3265, 3266, 3267, 3272, 3274, 3277, 3278, 3279, 3284, 3285, 3287, 3288, 3290, 3292, 3295, 3298, 3299, 3304, 3305, 3306, 3307, 3312, 3313, 3316, 3317, 3318, 3320, 3325, 3326, 3328, 3331, 3332, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3350, 3351, 3354, 3355, 3357, 3359, 3360, 3363, 3365, 3368, 3369, 3372, 3373, 3374, 3375, 3376, 3380, 3382, 3383, 3384, 3385, 3388, 3389, 3390, 3392, 3393, 3395, 3396, 3397, 3398, 3403, 3404, 3405, 3406, 3411, 3413, 3417, 3418, 3419, 3421, 3423, 3424, 3426, 3427, 3429, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3440, 3441, 3442, 3443, 3444, 3445, 3447, 3448, 3449, 3451, 3453, 3455, 3456, 3457, 3460, 3463, 3464, 3465, 3466, 3467, 3468, 3470, 3471, 3473, 3474, 3475, 3476, 3477, 3479, 3481, 3482, 3483, 3484, 3486, 3487, 3491, 3492, 3493, 3495, 3496, 3497, 3498, 3499, 3502, 3512, 3514, 3515, 3516, 3517, 3520, 3521, 3523, 3524, 3525, 3527, 3

### Fastex

In [2]:
!pip install fasttext

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 136.2 kB/s eta 0:00:0031m93.8 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=3416620 sha256=c5345999d4e9065db533ec83b748c11a709a57a1252d8641ccb1937a89ebef79
  Stored in directory: /home/mamadoo/.cache/pip/wheels/fc/22/93/1a3d535655339964fd8936d807ec85da466303d545023d2139
Successfully built fasttext
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [24]:
import fasttext
from numpy.linalg import norm
import numpy as np

In [5]:
fast = fasttext.load_model("cc.fa.300.bin")
dim = fast.get_dimension()

In [36]:
def create_doc_vectors(dataset):
    doc_vectors = {}
    counter = 0
    for data in dataset:
        doc_vector = np.zeros(dim)
        num_of_tokens = len(data['tokens'])
        for token in data['tokens']:
            doc_vector += fast.get_word_vector(token)
        doc_vector /= num_of_tokens
        norm = np.linalg.norm(doc_vector)
        doc_vector /=  norm
        doc_vectors[counter] = doc_vector
        counter += 1
    return doc_vectors

In [37]:
doc_vectors = create_doc_vectors(dataset)

In [41]:
def find_similars(query, fast_model, doc_vectors, dim, dataset, dataset_size):
    processed_query = preprocess_query(query)
    query_vector = np.zeros(dim)
    for token in processed_query:
        query_vector += fast_model.get_word_vector(token)
    query_vector /= len(processed_query)
    query_vector /= norm(query_vector)
    similar_docs = []
    for i in range(dataset_size):
        similar_docs.append((i, np.dot(doc_vectors[i], query_vector)))
    similar_docs.sort(key=lambda x: x[1], reverse=True)
    doc_ids = [x[0] for x in similar_docs]
    return doc_ids

#### Example 1

In [44]:
query = 'مهاجم فوتبال'
results = find_similars(query, fast, doc_vectors, dim, dataset, len(dataset))
show_first_ten_titles(results)

Top 10 results:
پرسپولیس با مهاجم خارجی به توافق رسید |‌ بمب زمستانی سرخ‌ها در آستانه انفجار
مهاجم برزیلی جانشین لوکادیا می شود؟ | رقم پیشنهاد سرخ‌ها به بازیکن گلزن مشخص شد
مشتری اروپایی مهاجم پرسپولیس مشخص شد 
مهاجم مدنظر پرسپولیس از باشگاهش جدا شد | سرخ‌ها در انتظار حضور در تهران
مهاجم برزیلی گزینه یحیی برای جانشینی لوکادیا | اتفاق عجیب در مذاکره سرخ ها با بازیکن خارجی
دعوت نامه کودتاگران به طارمی | وسوسه در جمع بزرگان
عکس| مهاجم برزیلی رسما به پرسپولیس پیوست
بمب خارجی پرسپولیس به زودی در تهران | سرخ ها آماده قرارداد با جانشین لوکادیا
۳ بازیکن پرسپولیس در لیست خروج یحیی | جانشین خارجی لوکادیا مشخص شد
همبازی سابق و رفیق مسی در یکقدمی تیم عربستانی | پیشنهاد رسمی رسید


#### Example 2

In [45]:
query = 'تحریم‌های ظالمانه'
results = find_similars(query, fast, doc_vectors, dim, dataset, len(dataset))
show_first_ten_titles(results)

Top 10 results:
منطق مذاکراتی ایران پذیرفته شده است | توصیف جالب معاون وزیرخارجه درباره ترامپ | مذاکرات برجام ادامه دارد؟
اتحادیه اروپا یک شرکت صنایع هواپیماسازی ایران را تحریم کرد | دلیل تحریم ظالمانه اعلام شد
واکنش ایران به تحریم‌های جدید انگلیس و اتحادیه اروپا | علیه مروجان خشونت اقدام متقابل می کنیم 
بورل: جایگزینی برای برجام وجود ندارد | کسانی که غیر از این فکر می‌کنند خود را فریب می‌دهند
واکنش آمریکا به روابط تجاری ایران و عراق | آیا بانک‌های عراقی تحریم می‌شوند؟
تبادل پیام‌های ایران و امریکا از کانال‌های مختلف | تهران - واشنگتن مذاکره مستقیم ندارند
استرالیا ۱۶ شخص و نهاد ایرانی را تحریم کرد | شخصیت‌های برجسته نیروی مقاومت بسیج در میان تحریم شدگان
تعدادی از شما حتی سفر کاری به اروپا نداشته‌اید و سرمایه‌ای آنجا ندارید ... | تصاویر معنادار وزیر خارجه ایران
 تاریخ انقضای برجام چه زمان است؟ | اتفاقی که از نظر غرب یعنی ایران به سلاح اتمی دست پیدا کرده است | آمریکا حق ندارد درباره مرگ برجام حرف بزند 
دفاع تمام قد مجلس از سپاه | هر مصوبه‌ای علیه سپاه با پاسخ متقابل مجلس ایران مواجه خواه

#### Example 3

In [46]:
query = 'ورزش کنید'
results = find_similars(query, fast, doc_vectors, dim, dataset, len(dataset))
show_first_ten_titles(results)

Top 10 results:
وزیر ورزش دومین کارت زرد را از مجلس گرفت | علت تذکر نمایندگان به سجادی
واکنش وزیر ورزش به تحریمش و حضور مجدد تماشاگران در ورزشگاه
واکنش به ماجرای تجاوز در مدرسه فوتبال مشهد |‌ مدیرکل ورزش و جوانان خراسان رضوی؛ به شدیدترین شکل برخورد می‌کنیم
نگاه واعظ آشتیانی به انتخابات کنفدراسیون فوتبال آسیا؛ یعنی باهوش تر و عاقل تر از شیخ سلمان وجود ندارد!؟ | این انتخابات مافیایی است
با رعایت این برنامه‌ها می‌توانید عمر طولانی‌تری داشته باشید | چگونه کلسترول خون را کاهش دهیم؟
سرنوشت کی روش به دست مجلس افتاد | استیضاح وزیر ورزش علیه تمدید قرارداد مرد پرتغالی
واکنش وزارت ورزش و جوانان به توهین ملی پوش سابق ایران به مقدسات 
مربی سرشناس جهان ورزش درگذشت
استیضاح وزیر ورزش با امضای ۵۱ نماینده کلید خورد
بازگشایی سه مجموعه ورزشی شهرداری در تهران 


#### Elastic Fuzzy Search

In [2]:
import json
import requests

In [3]:
# Elasticsearch endpoint configuration
ELASTICSEARCH_HOST = "192.168.57.2:8000"
ELASTICSEARCH_USER = "elastic"
ELASTICSEARCH_PASSWORD = "secret"
ELASTICSEARCH_ENDPOINT = "http://" + ELASTICSEARCH_USER + ':' + ELASTICSEARCH_PASSWORD + '@' + ELASTICSEARCH_HOST

In [4]:
INDEX_NAME = 'news_index'

In [5]:
def prepare_query(query, field):
    prepared_data = {
        "query": {
            "fuzzy": {
                field: {
                    "value": query
                }
            }
        }
    }
    return json.dumps(prepared_data)


In [6]:
def fuzzy_search(elasticsearch_endpoint, query, field, index_name):
    headers = {'Content-Type': 'application/json'}
    prepared_data = prepare_query(query, field)
    response = requests.get(f"{elasticsearch_endpoint}/{index_name}/_search/", data=prepared_data, headers=headers).content.decode()
    return json.loads(response)

#### Example 1

In [7]:
query = 'آمریگایی'
field = 'body'
fuzzy_search(ELASTICSEARCH_ENDPOINT, query, field, INDEX_NAME)

{'took': 111,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 269, 'relation': 'eq'},
  'max_score': 7.4145775,
  'hits': [{'_index': 'news_index',
    '_type': '_doc',
    '_id': 'NYu5bYYB2Omwr6TiR90_',
    '_score': 7.4145775,
    '_source': {'date': 'چهارشنبه ۵ بهمن ۱۴۰۱ - ۱۱:۰۲',
     'title': 'خوراکی\u200cهایی که در زمان باستان رواج داشتند | علت عجیب آدم\u200cخواری مردم گینه نو! | قهوه و لوبیا اولین بار در این نقاط کشت شدند',
     'intro': 'بی\u200cشک تاریخ و گذشته هر چیز برای ما جالب توجه است. پیشینه تولید و تکثیر محصولات زراعی \xa0نیز از این موضوع مستثنی نیست. به\u200cعنوان نمونه شاید برایتان جالب باشد پرکاربردترین مواد خوراکی مانند قهوه یا سیب\u200cزمینی از کجا آمده\u200cاند.',
     'body': 'به گزارش همشهری آنلاین، وقتی پایتان به طبیعت باز می\u200cشود بی\u200cشک میوه\u200cهای جنگلی را امتحان کرده\u200cاید. وقتی این میوه\u200cهای جنگلی را با میوه\u200cهایی که از فروشگاه\u200cها و تره\u200cبارهای محل زندگی خو

#### Example 2

In [8]:
query = 'فوتشال'
field = 'body'
fuzzy_search(ELASTICSEARCH_ENDPOINT, query, field, INDEX_NAME)

{'took': 121,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 496, 'relation': 'eq'},
  'max_score': 7.022976,
  'hits': [{'_index': 'news_index',
    '_type': '_doc',
    '_id': 'OYu5bYYB2Omwr6TiFdss',
    '_score': 7.022976,
    '_source': {'date': 'یکشنبه ۴ دی ۱۴۰۱ - ۰۸:۳۰',
     'title': 'ایران - برزیل مشروط شد | زنان هم باید بازی کنند! ',
     'intro': 'مذاکرات برای انجام بازی دوستانه تیم\u200cهای ملی فوتسال مردان ایران و برزیل در حالی ادامه دارد که برزیلی\u200cها خواهان بازی با تیم زنان کشورمان هم شده\u200cاند.',
     'body': 'به گزارش همشهری\u200cآنلاین، رایزنی فدراسیون فوتبال ایران و کمیته فوتسال با همتای برزیلی برای انجام بازی تدارکاتی در شهریور ۱۴۰۲ در کشور برزیل همچنان ادامه دارد. البته طرفین هنوز به صورت قطعی روی انجام این مسابقه و جزئیات آن به توافق نرسیده\u200cاند.\nپیگیری\u200cها نشان می\u200cدهد برزیلی\u200cها علاوه بر سفر تیم ملی مردان ایران، درخواست کرده\u200cاند تیم ملی زنان کشورمان هم عازم برزیل

#### Example 3

In [9]:
query = 'شلامت'
field = 'body'
fuzzy_search(ELASTICSEARCH_ENDPOINT, query, field, INDEX_NAME)

{'took': 23,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 316, 'relation': 'eq'},
  'max_score': 6.425227,
  'hits': [{'_index': 'news_index',
    '_type': '_doc',
    '_id': 'go65bYYBTzCi8CauQcjQ',
    '_score': 6.425227,
    '_source': {'date': 'جمعه ۱۶ دی ۱۴۰۱ - ۱۲:۲۲',
     'title': 'سرطانی که نشانه اش نفخ معده است | عوارض و نشانه\u200cهای نفخ زیاد برای بدن',
     'intro': 'نفخ یک مشکل گوارشی رایج است که بسیاری از افراد آن را تجربه می\u200cکنند؛ اگرچه در بسیاری از موارد، بدون هیچ اقدام خاصی این مشکل از بین می\u200cرود اما می\u200cتواند به یک معضل جدی برای سلامت هم تبدیل شود.',
     'body': 'به گزارش همشهری آنلاین، در برخی از شرایط نفخ دیگر یک علامت مزاحم نیست و می\u200cتواند نشانه\u200cهای نگران کننده\u200cای داشته باشد به طوری که نیاز به توجه فوری پزشکی را ایجاد کند.\nبنابر اعلام دفتر آموزش و ارتقای سلامت وزارت بهداشت، در صورتی که علائم زیر را به دنبال نفخ تجربه کردید حتما باید به پزشک مراجعه کنید:\n- درد ن